In [1]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
!pip install tiktoken

   ---------------------------------------- 0.0/214.0 kB ? eta -:--:--
   --------------------------------------- 214.0/214.0 kB 12.7 MB/s eta 0:00:00


In [100]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [101]:
!mkdir pdfs

A subdirectory or file pdfs already exists.


In [102]:
loader = PyPDFDirectoryLoader("pdfs") 

In [103]:
data = loader.load()

In [104]:
data[0]

Document(page_content='arXiv:1409.1556v6  [cs.CV]  10 Apr 2015Publishedasa conferencepaperat ICLR2015\nVERYDEEPCONVOLUTIONAL NETWORKS\nFORLARGE-SCALEIMAGERECOGNITION\nKarenSimonyan∗& AndrewZisserman+\nVisualGeometryGroup,DepartmentofEngineeringScience, UniversityofOxford\n{karen,az }@robots.ox.ac.uk\nABSTRACT\nIn this work we investigate the effect of the convolutional n etwork depth on its\naccuracy in the large-scale image recognition setting. Our main contribution is\na thorough evaluation of networks of increasing depth using an architecture with\nverysmall( 3×3)convolutionﬁlters,whichshowsthatasigniﬁcantimprove ment\non the prior-art conﬁgurations can be achieved by pushing th e depth to 16–19\nweight layers. These ﬁndings were the basis of our ImageNet C hallenge 2014\nsubmission,whereourteamsecuredtheﬁrstandthesecondpl acesinthelocalisa-\ntion and classiﬁcation tracks respectively. We also show th at our representations\ngeneralise well to other datasets, where they achieve stat

In [105]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)

In [106]:
text_chunks = text_splitter.split_documents(data)

In [107]:
text_chunks

[Document(page_content='arXiv:1409.1556v6  [cs.CV]  10 Apr 2015Publishedasa conferencepaperat ICLR2015\nVERYDEEPCONVOLUTIONAL NETWORKS\nFORLARGE-SCALEIMAGERECOGNITION\nKarenSimonyan∗& AndrewZisserman+\nVisualGeometryGroup,DepartmentofEngineeringScience, UniversityofOxford\n{karen,az }@robots.ox.ac.uk\nABSTRACT\nIn this work we investigate the effect of the convolutional n etwork depth on its\naccuracy in the large-scale image recognition setting. Our main contribution is', metadata={'source': 'pdfs\\1409.1556.pdf', 'page': 0}),
 Document(page_content='a thorough evaluation of networks of increasing depth using an architecture with\nverysmall( 3×3)convolutionﬁlters,whichshowsthatasigniﬁcantimprove ment\non the prior-art conﬁgurations can be achieved by pushing th e depth to 16–19\nweight layers. These ﬁndings were the basis of our ImageNet C hallenge 2014\nsubmission,whereourteamsecuredtheﬁrstandthesecondpl acesinthelocalisa-\ntion and classiﬁcation tracks respectively. We also show th 

In [108]:
len(text_chunks)

119

In [109]:
print(text_chunks[0].page_content)

arXiv:1409.1556v6  [cs.CV]  10 Apr 2015Publishedasa conferencepaperat ICLR2015
VERYDEEPCONVOLUTIONAL NETWORKS
FORLARGE-SCALEIMAGERECOGNITION
KarenSimonyan∗& AndrewZisserman+
VisualGeometryGroup,DepartmentofEngineeringScience, UniversityofOxford
{karen,az }@robots.ox.ac.uk
ABSTRACT
In this work we investigate the effect of the convolutional n etwork depth on its
accuracy in the large-scale image recognition setting. Our main contribution is


In [110]:
print(text_chunks[118].page_content)

whichimprovestheperformance.
v3Addsgeneralisationexperiments(AppendixB) on PASCAL VOC andCaltech image classiﬁca-
tiondatasets. Themodelsusedforthese experimentsarepub liclyavailable.
v4The paper is converted to ICLR-2015 submission format. Also adds experiments with multiple
cropsforclassiﬁcation.
v6Camera-readyICLR-2015conferencepaper. Addsa compariso nof the net B with a shallow net
andtheresultsonPASCAL VOCactionclassiﬁcationbenchmar k.
14


In [111]:
import os
os.environ["OPENAI_API_KEY"]=""

In [112]:
embedding = OpenAIEmbeddings()

In [113]:
embedding.embed_query("How are you")

[-0.016171867947776714,
 -0.00278568344715241,
 -0.004810338093806201,
 -0.03635182392278897,
 -0.023807526109539037,
 0.016615801453539348,
 -0.028995208614565993,
 -0.02004043268559754,
 -0.023388959966584976,
 -0.005130604541723888,
 0.02530421727905022,
 0.005593563569592951,
 -0.010521226880541953,
 0.003985889954963628,
 -0.01005826738701163,
 -0.01547425801996079,
 0.04375917209398405,
 -0.009259187076638892,
 0.008003489579099235,
 -0.014725912435205197,
 0.002363946616677909,
 0.009690436900997239,
 0.0006052559121646146,
 -0.002707995083643949,
 -0.016970948258149453,
 0.00015071943047532773,
 0.007121963942614852,
 -0.015677199716396934,
 0.004708867245588127,
 -0.02841175181938865,
 0.003307305549254133,
 -0.00549526434021784,
 -0.005790162959665694,
 0.0026350632170774107,
 0.007394665886774575,
 -0.021283446501732913,
 -0.00917674268184977,
 0.0030425307895567744,
 0.02185421868418345,
 0.005143288223128175,
 0.019837491920483917,
 -0.00011227160648902415,
 -0.00576479513

In [114]:
len(embedding.embed_query("How are you"))

1536

In [115]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '1eea3b44-8078-45af-9a31-8c18dbebe9bd')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [116]:
from pinecone import Pinecone

In [117]:
# GCP	us-west-1 (N. California)	us-west1-gcp
# GCP	us-central-1 (Iowa)	us-central1-gcp
# GCP	us-west-4 (Las Vegas)	us-west4-gcp
# GCP	us-east-4 (Virginia)	us-east4-gcp
# GCP	northamerica-northeast-1	northamerica-northeast1-gcp
# GCP	asia-northeast-1 (Japan)	asia-northeast1-gcp
# GCP	asia-southeast-1 (Singapore)	asia-southeast1-gcp
# GCP	us-east-1 (South Carolina)	us-east1-gcp
# GCP	eu-west-1 (Belgium)	eu-west1-gcp
# GCP	eu-west-4 (Netherlands)	eu-west4-gcp
# AWS	us-east-1 (Virginia)	us-east-1-aws
# Azure	eastus (Virginia)	eastus-azure

In [128]:
index_name = "test"

In [129]:
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

In [130]:
from pinecone import Pinecone, PodSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

vectorstore = pc.create_index(
  name=index_name,
  dimension=1536,
  metric="cosine",
  spec=PodSpec(
    environment="gcp-starter"
  )
)

In [132]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Create Embeddings for each of the Text Chunk

In [ ]:
docsearch = index.upsert([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [123]:
from langchain_community.vectorstores import Pinecone as PineconeLang

In [ ]:
docsearch

In [ ]:
docsearch.as_retriever()

In [ ]:
query= "YOLOv7 outperforms which models"

In [ ]:
docs = docsearch.similarity_search(query)

In [ ]:
docs

In [ ]:
llm = OpenAI()

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
query = "YOLOv7 outperforms which models"

In [ ]:
qa.run(query)

In [ ]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")